# Concise Implementation of Linear Regression

We will show you how to implement
the linear regression model
concisely by using high-level APIs

In [2]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

Call upon the existing API in a framework to read data

In [5]:
def load_array(data_arrays, batch_size, is_train=True):  
    """Construct a PyTorch data iterator."""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

[tensor([[-0.6178, -0.6908],
         [ 1.5681,  1.0766],
         [ 2.8053, -0.8076],
         [-1.0401, -1.5506],
         [-1.3891,  1.0033],
         [-0.4686,  0.3383],
         [ 0.1464,  0.7329],
         [-0.1767, -1.0826],
         [ 1.3438, -0.8969],
         [ 0.8134,  0.5363]]),
 tensor([[ 5.2928],
         [ 3.6691],
         [12.5433],
         [ 7.3812],
         [-1.9757],
         [ 2.1168],
         [ 2.0138],
         [ 7.5213],
         [ 9.9501],
         [ 4.0156]])]

Use a framework's predefined layers

In [6]:
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

Initialize the model parameters

In [7]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

The `MSELoss` class computes the mean squared error (without the $1/2$ factor in :eqref:`eq_mse`)

In [8]:
loss = nn.MSELoss()

Instantiate an `SGD` instance

In [9]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

The training loop itself is strikingly similar
to what we did when implementing everything from scratch

In [10]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000219
epoch 2, loss 0.000093
epoch 3, loss 0.000093


Compare the model parameters learned by training on finite data
and the actual parameters

In [11]:
w = net[0].weight.data
print('error in estimating w:', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('error in estimating b:', true_b - b)

error in estimating w: tensor([0.0003, 0.0004])
error in estimating b: tensor([-0.0003])
